In [8]:
from datasets import load_dataset,concatenate_datasets, Dataset,DatasetDict
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch

In [9]:
def collate(examples):
    # Convert to Tensors and build dataset

    input_ids_bert = pad_sequence([torch.tensor(f['bert_token'], dtype=torch.long) for f in examples],
                                  batch_first=True, padding_value=bert_pad_token)
    input_ids_gpt = pad_sequence([torch.tensor(f['gpt2_token'], dtype=torch.long) for f in examples],
                                 batch_first=True, padding_value=gpt2_pad_token)
    try:
        token_lengths = torch.tensor([[len(f['bert_token']), len(f['gpt2_token'])] for f in examples],
                                     dtype=torch.long)
    except:
        token_lengths = torch.zeros((len(examples), 1091))
        for i in range(len(examples)):
            token_lengths[i, len(examples[i]['gpt2_token'])] = 1
    return (input_ids_bert, input_ids_gpt, token_lengths)

In [10]:
train_eval_dataset =load_dataset("guangyil/yelp_short_v2")
eval_dataloader =  DataLoader(train_eval_dataset['test'], num_workers=0, collate_fn=collate,batch_size=64)

In [15]:
tokenizer_encoder = AutoTokenizer.from_pretrained("prajjwal1/bert-small")
tokenizer_decoder = AutoTokenizer.from_pretrained("gpt2-xl")

special_tokens_dict = {'pad_token': '<PAD>', 'bos_token': '<BOS>', 'eos_token': '<EOS>', }
num_added_toks = tokenizer_decoder.add_special_tokens(special_tokens_dict)
gpt2_pad_token = tokenizer_decoder.convert_tokens_to_ids([tokenizer_decoder.pad_token])[0]
gpt2_eos_token = tokenizer_decoder.convert_tokens_to_ids([tokenizer_decoder.eos_token])[0]
gpt2_bos_token = tokenizer_decoder.convert_tokens_to_ids([tokenizer_decoder.bos_token])[0]
bert_pad_token = tokenizer_encoder.pad_token_id


In [12]:
from models.my_transformers import *
from models.models import VAE
from train.reconstruction import *

In [13]:
encoder = MODEL_CLASS['BertForLatentConnectorAVG']
decoder = MODEL_CLASS['GPT2ForLatentConnectorNew']
latent_size = 64
model_encoder = encoder.from_pretrained("prajjwal1/bert-small", latent_size=latent_size,
                                                    pad_id=tokenizer_encoder.pad_token_id,local_files_only=False)
model_decoder = decoder.from_pretrained("bert-base-cased", latent_size = latent_size,latent_as_gpt_emb=True,
                                                    latent_as_gpt_memory=True,local_files_only=False)

output_dir = "test"
model_vae = VAE(model_encoder, model_decoder, tokenizer_encoder, tokenizer_decoder, latent_size, output_dir)

Some weights of BertForLatentConnectorAVG were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['bert.linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are using a model of type bert to instantiate a model of type gpt2. This is not supported for all configurations of models and can yield errors.
Some weights of GPT2ForLatentConnectorNew were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['h.12.attn.c_proj.weight', 'h.2.ln_1.weight', 'h.10.mlp.c_proj.bias', 'h.9.attn.c_attn.weight', 'h.3.attn.c_attn.bias', 'h.11.attn.c_attn.weight', 'h.12.attn.c_proj.bias', 'h.11.attn.c_proj.bias', 'h.9.attn.c_attn.bias', 'h.2.ln_2.bias', 'h.0.ln_1.weight', 'h.0.mlp.c_fc.bias', 'h.11.ln_2.weight', 'h.11.mlp.c_fc.bias', 'h.2.ln_2.weight', 'h.0.mlp.c_fc.weight', 'h.5.mlp.c_fc.weight', 'h.12.ln_2.bias', 'h.4.mlp.c_fc.weight', 'h.8.ln_1.weigh

In [16]:
calc_rec_lgy(model_vae, tokenizer_encoder, tokenizer_decoder,eval_dataloader, "cpu", True, ns=1)

IndexError: index out of range in self

In [35]:
tqdm(eval_dataloader)

  0%|          | 0/16 [00:00<?, ?it/s]

In [34]:
from tqdm import tqdm